# The While language

The While language is a procedural language that is commonly used for learning program analysis. Is the barebones of an imperative language that can be extended for reasoning about other properties of programs.

The abstract syntax of the While language is as follows

In [1]:
type var = string
type label = int

type expr = Var of var 
          | Const of int
          | Op of string * expr * expr
type bool_expr = True | False
               | Not of bool_expr
               | BoolOp of string * bool_expr * bool_expr
               | RelOp of string * expr * expr
type stmt = Skip of label
          | Assign of var * expr * label
          | Seq of stmt * stmt
          | If of bool_expr * label * stmt * stmt
          | While of bool_expr * label * stmt

type var = string


type label = int


type expr = Var of var | Const of int | Op of string * expr * expr


type bool_expr =
    True
  | False
  | Not of bool_expr
  | BoolOp of string * bool_expr * bool_expr
  | RelOp of string * expr * expr


type stmt =
    Skip of label
  | Assign of var * expr * label
  | Seq of stmt * stmt
  | If of bool_expr * label * stmt * stmt
  | While of bool_expr * label * stmt


We can define an example program with this syntax:

```python
y = x          # 1
z = 1          # 2
while y > 1:   # 3
    z = z * y  # 4
    y = y - 1  # 5
y = 0          # 6
```

Each number represent the label of that part of the program. In OCaml syntax this program will look like this

In [2]:
module Examples = struct
    let factorial = Seq (
                            Assign ("y", (Var "x"), 1),
                            Seq (
                                    Assign ("z", (Const 1), 2),
                                    Seq (
                                            While (
                                                    RelOp (">", (Var "y"), (Const 1)), 
                                                    3, 
                                                    Seq (
                                                            Assign ("z", Op ("*", (Var "z"), (Var "y")), 4),
                                                            Assign ("y", Op ("-", (Var "y"), (Const 1)), 5)
                                                        )
                                                  ),
                                            Assign ("y", (Const 0), 6)
                                        )
                                )
                        )
end

module Examples : sig val factorial : stmt end


For the analysis of the programs we will need mostly three things from the program:

- The variables in the program
- The labels defined in the program
- The call graph

We can write some utility functions that will give us this information

In [3]:
module VarsSet = Set.Make(String)

let rec vars_of_expr = function
| Var v -> VarsSet.singleton v
| Const _ -> VarsSet.empty
| Op (_, e1, e2) -> VarsSet.union (vars_of_expr e1) (vars_of_expr e2)

let rec vars_of_bool_expr = function
| True | False -> VarsSet.empty
| Not b -> vars_of_bool_expr b
| BoolOp (_, b1, b2) -> VarsSet.union (vars_of_bool_expr b1) (vars_of_bool_expr b2)
| RelOp (_, e1, e2) -> VarsSet.union (vars_of_expr e1) (vars_of_expr e2)

let rec vars_of_stmt = function
| Skip _ -> VarsSet.empty
| Assign (v, e, _) -> VarsSet.union (VarsSet.singleton v) (vars_of_expr e)
| Seq (s1, s2) -> VarsSet.union (vars_of_stmt s1) (vars_of_stmt s2)
| If (b, _, s1, s2) -> VarsSet.union (VarsSet.union (vars_of_bool_expr b) (vars_of_stmt s1)) (vars_of_stmt s2)
| While (b, _, s) -> VarsSet.union (vars_of_bool_expr b) (vars_of_stmt s)

module LabelsSet = Set.Make(struct 
   type t = int 
   let compare = compare
end)
 
let rec labels_of_stmt = function
| Skip l -> LabelsSet.singleton l
| Assign (_, _, l) -> LabelsSet.singleton l
| Seq (s1, s2) -> LabelsSet.union (labels_of_stmt s1) (labels_of_stmt s2)
| If (_, l, s1, s2) -> LabelsSet.union (LabelsSet.union (LabelsSet.singleton l) (labels_of_stmt s1)) (labels_of_stmt s2)
| While (_, l, s) -> LabelsSet.union (LabelsSet.singleton l) (labels_of_stmt s)


module VarsSet :
  sig
    type elt = String.t
    type t = Set.Make(String).t
    val empty : t
    val is_empty : t -> bool
    val mem : elt -> t -> bool
    val add : elt -> t -> t
    val singleton : elt -> t
    val remove : elt -> t -> t
    val union : t -> t -> t
    val inter : t -> t -> t
    val diff : t -> t -> t
    val compare : t -> t -> int
    val equal : t -> t -> bool
    val subset : t -> t -> bool
    val iter : (elt -> unit) -> t -> unit
    val map : (elt -> elt) -> t -> t
    val fold : (elt -> 'a -> 'a) -> t -> 'a -> 'a
    val for_all : (elt -> bool) -> t -> bool
    val exists : (elt -> bool) -> t -> bool
    val filter : (elt -> bool) -> t -> t
    val partition : (elt -> bool) -> t -> t * t
    val cardinal : t -> int
    val elements : t -> elt list
    val min_elt : t -> elt
    val min_elt_opt : t -> elt option
    val max_elt : t -> elt
    val max_elt_opt : t -> elt option
    val choose : t -> elt
    val choose_opt : t -> elt option
    val split : 

val vars_of_expr : expr -> VarsSet.t = <fun>


val vars_of_bool_expr : bool_expr -> VarsSet.t = <fun>


val vars_of_stmt : stmt -> VarsSet.t = <fun>


module LabelsSet :
  sig
    type elt = int
    type t
    val empty : t
    val is_empty : t -> bool
    val mem : elt -> t -> bool
    val add : elt -> t -> t
    val singleton : elt -> t
    val remove : elt -> t -> t
    val union : t -> t -> t
    val inter : t -> t -> t
    val diff : t -> t -> t
    val compare : t -> t -> int
    val equal : t -> t -> bool
    val subset : t -> t -> bool
    val iter : (elt -> unit) -> t -> unit
    val map : (elt -> elt) -> t -> t
    val fold : (elt -> 'a -> 'a) -> t -> 'a -> 'a
    val for_all : (elt -> bool) -> t -> bool
    val exists : (elt -> bool) -> t -> bool
    val filter : (elt -> bool) -> t -> t
    val partition : (elt -> bool) -> t -> t * t
    val cardinal : t -> int
    val elements : t -> elt list
    val min_elt : t -> elt
    val min_elt_opt : t -> elt option
    val max_elt : t -> elt
    val max_elt_opt : t -> elt option
    val choose : t -> elt
    val choose_opt : t -> elt option
    val split : elt -> t -> t * bool * t

val labels_of_stmt : stmt -> LabelsSet.t = <fun>


## CFG generation

The CFG can be generated by induction. The base cases are the rules in the abstract syntax that dont have recursive rules for staments.

A CFG is composed of a set of nodes, a set of vertices, the begin node and the end node

$ f : (G, p), s \rightarrow (G, p) $ is the function that generates the graph given a graph and the current statement and $ G \equiv (n_0, v_0, b_0, e_0) $ 

### Base cases

$$ f(((n_0, v_0, b_0, e_0), p_0), \mathbf{Skip}\,l) = ((n_0 \vee \{\mathtt{skip}\,l\}, v_0 \vee \{(\mathtt{skip}\, l, e_0)\} \vee \{(p, \mathtt{skip}\, l) | p \in p_0\} / \{(p, e_0) | p \in p_0\}, b_0, e_0), \{\mathtt{skip}\,l\}) $$

$$ f(((n_0, v_0, b_0, e_0), p_0), \mathbf{Assign}\,(v,e,l)) = ((n_0 \vee \{\mathtt{assign}\,(v,e,l)\}, v_0 \vee \{(\mathtt{assign}\, (v,e,l), e_0)\} \vee \{(p, \mathtt{assign}\, (v,e,l)) | p \in p_0\} / \{(p, e_0) | p \in p_0\}, b_0, e_0), \{\mathtt{assign}\,(v,e,l)\}) $$


### Recursive cases

$$ f(((n_0, v_0, b_0, e_0), p_0), \mathbf{Seq}\,(s_1, s_2)) = f(f(((n_0, v_0, b_0, e_0), p_0), s_1), s_2) $$

$$ f(((n_0, v_0, b_0, e_0), p_0), \mathbf{If}\,(b, l, s_1, s_2)) = ((n_3, v_3, b_0, e_0), p_1 \vee p_2);\\ n_1 = n_0 \vee \{\mathtt{bool}\, (b, l)\},\\ v_1 = v_0 \vee \{(\mathtt{bool}\, (b, l), e_0)\} \vee \{(p, \mathtt{bool}\, (b, l)) | p \in p_0\} / \{(p, e_0) | p \in p_0\},\\ ((n_2, v_2, b_2, e_2), p_1) = f(((n_1, v_1, b_0, e_0), \{\mathtt{bool}\, (b, l)\}), s_1),\\ ((n_3, v_3, b_3, e_3), p_2) = f(((n_2, v_2, b_0, e_0), \{\mathtt{bool}\, (b, l)\}), s_2) $$

$$ f(((n_0, v_0, b_0, e_0), p_0), \mathbf{While}\,(b, l, s)) = ((n_1, v_1 \vee \{(p, \mathtt{bool}\, (b, l)) | p \in p_1\} / \{(p, e_0) | p \in p_1\}, b_0, e_0), \{\mathtt{bool}\, (b, l)\});\\ ((n_1, v_1, b_1, e_1), p_1) = f(((n_0 \vee \{\mathtt{bool}\, (b, l)\},
                                            v_0 \vee \{(\mathtt{bool}\, (b, l), e_0)\} \vee \{(p, \mathtt{bool}\, (b, l)) | p \in p_0\} / \{(p, e_0) | p \in p_0\},
                                            b_0,
                                            e_0),
                                            \{\mathtt{bool}\, (b, l)\}), s) $$



In [4]:
type graph_node = BeginNode | EndNode 
                | SkipNode of label
                | AssignNode of var * expr * label
                | BoolNode of bool_expr * label
type graph_edge = graph_node * graph_node

module CFGNodes = Set.Make(struct
    type t = graph_node
    let compare = compare
end)

module CFGEdges = Set.Make(struct
    type t = graph_edge
    let compare = compare
end)

type cfg = CFGNodes.t * CFGEdges.t * graph_node * graph_node

let empty_cfg = (
    CFGNodes.of_list [BeginNode; EndNode], 
    CFGEdges.singleton (BeginNode, EndNode), 
    BeginNode, 
    EndNode)

let pnl n1 n2 = (n2, n1)
let nodes_to_edges f n = CFGNodes.to_seq n |> Seq.map f |> CFGEdges.of_seq
let add_node nodes node = CFGNodes.add node nodes
let add_vertex vertices vertex = CFGEdges.add vertex vertices
let connect p0 node n0 = nodes_to_edges (pnl node) p0 |> CFGEdges.union n0
let not_in_set set elt = CFGEdges.mem elt set |> not
let disconnect p0 node n0 = let pairs = nodes_to_edges (pnl node) p0 in 
                            CFGEdges.filter (not_in_set pairs) n0

let rec _build_cfg ((n0, v0, b0, e0), p0) = function
| Skip l -> let node = SkipNode l in ((
                add_node n0 node,
                add_vertex v0 (node, e0) |> connect p0 node |> disconnect p0 e0,
                b0, e0
            ), CFGNodes.singleton node)
| Assign (v, e, l) -> let node = AssignNode (v, e, l) in ((
                          add_node n0 node,
                          add_vertex v0 (node, e0) |> connect p0 node |> disconnect p0 e0,
                          b0, e0
                      ), CFGNodes.singleton node)
| Seq (s1, s2) -> _build_cfg (_build_cfg ((n0, v0, b0, e0), p0) s1) s2
| If (b, l, s1, s2) -> let node = BoolNode (b, l) in
                       let n1 = add_node n0 node and
                           v1 = add_vertex v0 (node, e0) |> connect p0 node |> disconnect p0 e0 in
                       let ((n2, v2, _, _), p1) = _build_cfg ((n1, v1, b0, e0), (CFGNodes.singleton node)) s1 in
                       let ((n3, v3, _, _), p2) = _build_cfg ((n2, v2, b0, e0), (CFGNodes.singleton node)) s2 in
                       ((n3, v3, b0, e0), CFGNodes.union p1 p2)
| While (b, l, s) -> let node = BoolNode (b, l) in
                     let ((n1, v1, _, _), p1) = _build_cfg ((
                                                     add_node n0 node,
                                                     add_vertex v0 (node, e0) |> connect p0 node |> disconnect p0 e0,
                                                     b0, e0
                                                ), (CFGNodes.singleton node)) s in
                     ((n1, v1 |> connect p1 node |> disconnect p1 e0, b0, e0), (CFGNodes.singleton node))

let fst_elt_of_tuple (a, _) = a

let build_cfg stmt = let (n0, v0, b0, e0) = empty_cfg in 
                     _build_cfg ((n0, v0, b0, e0), CFGNodes.singleton b0) stmt |> fst_elt_of_tuple


type graph_node =
    BeginNode
  | EndNode
  | SkipNode of label
  | AssignNode of var * expr * label
  | BoolNode of bool_expr * label


type graph_edge = graph_node * graph_node


module CFGNodes :
  sig
    type elt = graph_node
    type t
    val empty : t
    val is_empty : t -> bool
    val mem : elt -> t -> bool
    val add : elt -> t -> t
    val singleton : elt -> t
    val remove : elt -> t -> t
    val union : t -> t -> t
    val inter : t -> t -> t
    val diff : t -> t -> t
    val compare : t -> t -> int
    val equal : t -> t -> bool
    val subset : t -> t -> bool
    val iter : (elt -> unit) -> t -> unit
    val map : (elt -> elt) -> t -> t
    val fold : (elt -> 'a -> 'a) -> t -> 'a -> 'a
    val for_all : (elt -> bool) -> t -> bool
    val exists : (elt -> bool) -> t -> bool
    val filter : (elt -> bool) -> t -> t
    val partition : (elt -> bool) -> t -> t * t
    val cardinal : t -> int
    val elements : t -> elt list
    val min_elt : t -> elt
    val min_elt_opt : t -> elt option
    val max_elt : t -> elt
    val max_elt_opt : t -> elt option
    val choose : t -> elt
    val choose_opt : t -> elt option
    val split : elt -> t -> t * bo

module CFGEdges :
  sig
    type elt = graph_edge
    type t
    val empty : t
    val is_empty : t -> bool
    val mem : elt -> t -> bool
    val add : elt -> t -> t
    val singleton : elt -> t
    val remove : elt -> t -> t
    val union : t -> t -> t
    val inter : t -> t -> t
    val diff : t -> t -> t
    val compare : t -> t -> int
    val equal : t -> t -> bool
    val subset : t -> t -> bool
    val iter : (elt -> unit) -> t -> unit
    val map : (elt -> elt) -> t -> t
    val fold : (elt -> 'a -> 'a) -> t -> 'a -> 'a
    val for_all : (elt -> bool) -> t -> bool
    val exists : (elt -> bool) -> t -> bool
    val filter : (elt -> bool) -> t -> t
    val partition : (elt -> bool) -> t -> t * t
    val cardinal : t -> int
    val elements : t -> elt list
    val min_elt : t -> elt
    val min_elt_opt : t -> elt option
    val max_elt : t -> elt
    val max_elt_opt : t -> elt option
    val choose : t -> elt
    val choose_opt : t -> elt option
    val split : elt -> t -> t * bo

type cfg = CFGNodes.t * CFGEdges.t * graph_node * graph_node


...

val not_in_set : CFGEdges.t -> CFGEdges.elt -> bool = <fun>


val disconnect : CFGNodes.t -> graph_node -> CFGEdges.t -> CFGEdges.t = <fun>


val _build_cfg :
  (CFGNodes.t * CFGEdges.t * 'a * graph_node) * CFGNodes.t ->
  stmt -> (CFGNodes.t * CFGEdges.t * 'a * graph_node) * CFGNodes.t = <fun>


val fst_elt_of_tuple : 'a * 'b -> 'a = <fun>


val build_cfg : stmt -> CFGNodes.t * CFGEdges.t * graph_node * graph_node =
  <fun>


In [5]:
let rec string_of_expr = function
| Var v -> v
| Const c -> string_of_int c
| Op (o, e1, e2) -> Printf.sprintf "(%s %s %s)" (string_of_expr e1) o (string_of_expr e2)

let rec string_of_bool_expr = function
| True -> "True"
| False -> "False"
| Not b -> Printf.sprintf "!%s" (string_of_bool_expr b)
| BoolOp (o, b1, b2) -> Printf.sprintf "(%s %s %s)" (string_of_bool_expr b1) o (string_of_bool_expr b2)
| RelOp (o, e1, e2) -> Printf.sprintf "(%s %s %s)" (string_of_expr e1) o (string_of_expr e2)

let string_of_cfgnode = function
| BeginNode -> "begin"
| EndNode -> "end"
| AssignNode (v, e, l) -> Printf.sprintf "\"[%s := %s] %d\"" v (string_of_expr e) l
| SkipNode l -> Printf.sprintf "\"[skip] %d\"" l
| BoolNode (b, l) -> Printf.sprintf "\"[%s] %d\"" (string_of_bool_expr b) l

let print_relation (f, t) =
    "  " ^ (string_of_cfgnode f) ^ " -> " ^ (string_of_cfgnode t) ^ "\n"

let cfg_dotfile (_, v, _, _) =
    "digraph {\n" ^
    (CFGEdges.to_seq v |> Seq.map print_relation |> Seq.fold_left (^) "" ) ^
    "}\n"

val string_of_expr : expr -> var = <fun>


val string_of_bool_expr : bool_expr -> string = <fun>


val string_of_cfgnode : graph_node -> string = <fun>


val print_relation : graph_node * graph_node -> string = <fun>


val cfg_dotfile : 'a * CFGEdges.t * 'b * 'c -> string = <fun>


In [6]:
build_cfg Examples.factorial |> cfg_dotfile |> print_endline


digraph {
  begin -> "[y := x] 1"
  "[y := x] 1" -> "[z := 1] 2"
  "[y := 0] 6" -> end
  "[y := (y - 1)] 5" -> "[(y > 1)] 3"
  "[z := 1] 2" -> "[(y > 1)] 3"
  "[z := (z * y)] 4" -> "[y := (y - 1)] 5"
  "[(y > 1)] 3" -> "[y := 0] 6"
  "[(y > 1)] 3" -> "[z := (z * y)] 4"
}



- : unit = ()
